<a href="https://colab.research.google.com/github/gparedesg/proyecto-integrador/blob/master/Embeddings%2BCFDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers

## En la iteración anterior, el modelo presentó un desempeño limitado con una precisión del 34%. Para abordar esta limitación, hemos implementado las siguientes acciones.


### **Optimización mediante Embeddings**

Implementamos un enfoque basado en embeddings, donde inicialmente buscamos recomendaciones minimizando la distancia al primer resultado. Sin embargo, este enfoque no ofreció una precisión óptima. Por ello, decidimos expandir la búsqueda a las 10 recomendaciones más cercanas, con el objetivo de proporcionar al usuario un conjunto más amplio de opciones relevantes. Aunque los resultados han mejorado, seguimos considerando posibles ajustes al modelo o la integración de otros algoritmos que puedan optimizar aún más la precisión de las recomendaciones.

###**Ampliación del DataFrame para Generación de Embeddings**

Extendimos el DataFrame original mediante la columna ‘similar’, para generar más embeddings que permitan capturar de manera más precisa las características descriptivas y semánticas de los datos. Esta estrategia busca incrementar las opciones de recomendación, alineándolas mejor con las descripciones originales proporcionadas por los usuarios.

### **Optimización de la Normalización de Entradas**

Mantenemos el mismo enfoque en términos de estructura del código y parámetros de “units”, ya que consideramos que esta configuración es adecuada. Sin embargo, seguimos iterando y afinando estos componentes para maximizar el rendimiento.

### **Normalización y Estandarización del Lenguaje de Usuario**

En esta fase, buscamos combinar la entrada del usuario en una estructura uniforme, por ejemplo: “Vendí 3 bolsas de X kilos de comida para perro”. Esta normalización se realiza para segmentar los componentes de la oración y estructurarlos de manera que el modelo pueda procesarlos y categorizarlos eficientemente. Además, habilitamos la posibilidad de que el usuario ingrese componentes por separado, lo cual, con un catálogo más acotado y un lenguaje simplificado, ha demostrado mejorar la precisión de las recomendaciones generadas.



Adjuntamos los resultados obtenidos hasta el momento y continuamos trabajando en nuevas iteraciones para optimizar el modelo.

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sentence_transformers import SentenceTransformer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [ ]:
# Intentar cargar el CSV expandido
expanded_file_path = '/content/cfdi_expanded.csv'
try:
    df = pd.read_csv(expanded_file_path)
    print("Archivo expandido cargado correctamente.")
except FileNotFoundError:
    # Si no se encuentra el archivo expandido, cargar el archivo original y expandir las descripciones
    file_path = '/content/cfdi.csv'
    df = pd.read_csv(file_path)

    # Asegurar que las columnas necesarias existen en el DataFrame
    if 'description' not in df.columns or 'code' not in df.columns:
        raise ValueError("El CSV debe tener columnas 'description' y 'code'")

    # Extender el DataFrame con las descripciones alternas
    def expandir_descripciones(df):
        rows = []
        for _, row in df.iterrows():
            description = row['description']
            # Agregar la descripción original como 'usable_desc'
            new_row = row.copy()
            new_row['usable_desc'] = description
            rows.append(new_row)
            # Agregar descripciones alternas si existen
            similar_descriptions = row['similar'].split(',') if 'similar' in df.columns and pd.notna(row['similar']) else []
            for desc in similar_descriptions:
                new_row = row.copy()
                new_row['usable_desc'] = desc.strip()
                rows.append(new_row)
        return pd.DataFrame(rows)

    df = expandir_descripciones(df)
    # Guardar el DataFrame expandido en un nuevo archivo CSV
    df.to_csv(expanded_file_path, index=False)
    print("Archivo expandido creado y guardado como 'cfdi_expanded.csv'.")

Archivo expandido creado y guardado como 'cfdi_expanded.csv'.


In [ ]:
df.head(10)

,code,description,similar,type,division,group,usable_desc
0,1010101,No existe en el catálogo,Público en general,NaN,NaN,NaN,No existe en el catálogo
0,1010101,No existe en el catálogo,Público en general,NaN,NaN,NaN,Público en general
1,10101500,Animales vivos de granja,NaN,Productos,"Material Vivo Vegetal y Animal, Accesorios y S...",Animales vivos,Animales vivos de granja
2,10101501,Gatos vivos,NaN,Productos,"Material Vivo Vegetal y Animal, Accesorios y S...",Animales vivos,Gatos vivos
3,10101502,Perros,NaN,Productos,"Material Vivo Vegetal y Animal, Accesorios y S...",Animales vivos,Perros
4,10101504,Visón,NaN,Productos,"Material Vivo Vegetal y Animal, Accesorios y S...",Animales vivos,Visón
5,10101505,Ratas,NaN,Productos,"Material Vivo Vegetal y Animal, Accesorios y S...",Animales vivos,Ratas
6,10101506,Caballos,"Equinos, Potrancas, Potras, Potrillos, Potros,...",Productos,"Material Vivo Vegetal y Animal, Accesorios y S...",Animales vivos,Caballos
6,10101506,Caballos,"Equinos, Potrancas, Potras, Potrillos, Potros,...",Productos,"Material Vivo Vegetal y Animal, Accesorios y S...",Animales vivos,Equinos
6,10101506,Caballos,"Equinos, Potrancas, Potras, Potrillos, Potros,...",Productos,"Material Vivo Vegetal y Animal, Accesorios y S...",Animales vivos,Potrancas


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
stopwords_es = set(stopwords.words('spanish'))
'''
stopwords_es.add('suministros')
stopwords_es.add('materiales')
stopwords_es.add('equipos')
stopwords_es.add('servicios')
stopwords_es.add('maquinaria')
stopwords_es.add('componentes')
stopwords_es.add('vivo')
'''

def preprocesamiento_texto(texto):
  if texto:
    texto_sin_puntuacion = re.sub(r'[^\w\s]', '', texto)
    palabras = word_tokenize(texto_sin_puntuacion)
    palabras_filtradas = [palabra for palabra in palabras if palabra.lower() not in stopwords_es]
    return str(" ".join(palabras_filtradas))
  else:
    return ""

df['usable_desc'] = df['usable_desc'].apply(preprocesamiento_texto)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Importando el modelo para calcular los embeddings
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')

# Calcular los embeddings usando el usable description
descriptions = df['usable_desc'].tolist()
description_embeddings = model.encode(descriptions, convert_to_tensor=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Búsqueda de producto basada en Cosine Similarity
def encontrar_opciones_producto(producto, n=5):
    # Convertir entrada a embedding
    producto_embedding = model.encode([producto], convert_to_tensor=True)

    # Mover los tensores a la CPU antes de convertirlos a arrays de NumPy
    producto_embedding_cpu = producto_embedding.cpu()
    description_embeddings_cpu = description_embeddings.cpu()

    # Calcular la similitud del coseno entre el producto y las descripciones
    similidades = cosine_similarity(producto_embedding_cpu.numpy(), description_embeddings_cpu.numpy())[0]

    # Obtener los índices de las descripciones más similares
    indices_mas_similares = similidades.argsort()[-n:][::-1]

    # Obtener los códigos y las descripciones de los productos más similares
    resultados = [(df.iloc[indice]['code'], df.iloc[indice]['description']) for indice in indices_mas_similares]

    return resultados

# Búsqueda de producto basada en Euclidian Distances
def encontrar_opciones_producto_euclidiana(producto, n=5):
    # Convertir entrada a embedding
    producto_embedding = model.encode([producto], convert_to_tensor=True)

    # Mover los tensores a la CPU antes de convertirlos a arrays de NumPy
    producto_embedding_cpu = producto_embedding.cpu()
    description_embeddings_cpu = description_embeddings.cpu()

    # Calcular la distancia euclidiana entre el producto y las descripciones
    distancias = euclidean_distances(producto_embedding_cpu.numpy(), description_embeddings_cpu.numpy())[0]

    # Obtener los índices de las descripciones con menor distancia
    indices_mas_similares = distancias.argsort()[:n]

    # Obtener los códigos y las descripciones de los productos más similares
    resultados = [(df.iloc[indice]['code'], df.iloc[indice]['description']) for indice in indices_mas_similares]

    return resultados

In [ ]:
# Probando Cosine
producto = "comida para mascotas"
producto_clean = preprocesamiento_texto(producto)
recomendaciones = 10

print("USANDO COSINE SIMULARITY")
print(f'Entrada: {producto}')
resultados_encontrados = encontrar_opciones_producto(producto_clean, n=recomendaciones)
for i, (codigo, descripcion) in enumerate(resultados_encontrados, start=1):
    print(f'Resultado {i}: Código: {codigo}, Descripción: "{descripcion}"')

print("\n")

# Probando Euclidian
print("USANDO EUCLIDIAN DISTANCE")
print(f'Entrada: {producto}')
resultados_encontrados_euclidiana = encontrar_opciones_producto_euclidiana(producto_clean, n=recomendaciones)
for i, (codigo, descripcion) in enumerate(resultados_encontrados_euclidiana, start=1):
    print(f'Resultado Euclidiana {i}: Código: {codigo}, Descripción: "{descripcion}"')

USANDO COSINE SIMULARITY
Entrada: comida para mascotas
Resultado 1: Código: 10111304, Descripción: "Tazones o equipo para alimentación de mascotas"
Resultado 2: Código: 10122103, Descripción: "Comida para monos"
Resultado 3: Código: 10121806, Descripción: "Botanas o comida recreacional para gatos o perros"
Resultado 4: Código: 10122100, Descripción: "Comida para animales variados"
Resultado 5: Código: 10121804, Descripción: "Comida seca para gatos"
Resultado 6: Código: 10111303, Descripción: "Equipo para el manejo de desperdicios de las mascotas"
Resultado 7: Código: 10131600, Descripción: "Recipientes para animales"
Resultado 8: Código: 10122104, Descripción: "Comida para conejos"
Resultado 9: Código: 10121801, Descripción: "Comida seca para perros"
Resultado 10: Código: 10121800, Descripción: "Alimento para perros y gatos"


USANDO EUCLIDIAN DISTANCE
Entrada: comida para mascotas
Resultado Euclidiana 1: Código: 10122103, Descripción: "Comida para monos"
Resultado Euclidiana 2: Código